In [1]:
import os
import pygame
import time
from tkinter import Tk, Label, Button, simpledialog, messagebox, StringVar, OptionMenu

# Initialize pygame for playing audio
pygame.mixer.init()

# Base path to the audio files
AUDIO_BASE_PATH = os.path.join(os.getcwd(), "audio_files")

# Dictionary to map language and voice to the folder structure
VOICE_PATHS = {
    "EN": {"Hiva": "english/hiva", "MH": "english/mh", "Stella": "english/stella"},
    "FA": {"Hiva": "farsi/hiva", "MH": "farsi/mh"},
    "JP": {"Stella": "japanese/stella"}
}

# Global variables to store user choices
selected_language = None
selected_voice = None
BASE_AUDIO_PATH = None

# Function to play audio segments based on the file name
def play_audio(filename):
    pygame.mixer.music.load(os.path.join(BASE_AUDIO_PATH, f"{filename}.wav"))
    pygame.mixer.music.play()
    time.sleep(1)  # Adjust as necessary for the length of the audio file

# Function to handle playing minutes for a given hour, minute, and period in English
def play_time_audio_en(hour, minute, period):
    play_audio("its_now")
    
    if minute == 15:
        play_audio("a_quarter_past")
        play_audio(f"{hour}")
        play_audio(period)
    elif minute == 30:
        play_audio("half_past")
        play_audio(f"{hour}")
        play_audio(period)
    elif minute == 45:
        next_hour = (hour % 12) + 1
        play_audio("a_quarter_to")
        play_audio(f"{next_hour}")
        play_audio(period)
    else:
        play_audio(f"{hour}")
        
        if 11 <= minute <= 14 or 16 <= minute <= 19:
            play_audio(f"{minute}")  # Directly say the whole minute (e.g., "thirteen")
        else:
            tens = (minute // 10) * 10
            ones = minute % 10
            if tens > 0:
                play_audio(f"{tens}")
            if ones > 0:
                play_audio(f"{ones}")
        
        play_audio(period)       

# Time-telling logic for Farsi (with "and" logic)
def play_time_audio_fa(hour, minute, period):
    play_audio("its_now")

    if minute == 15:
        play_audio("a_quarter_past")
        play_audio(f"{hour}")
        play_audio(period)
    elif minute == 30:
        play_audio("half_past")
        play_audio(f"{hour}")
        play_audio(period)
    elif minute == 45:
        next_hour = (hour % 12) + 1
        play_audio("a_quarter_to")
        play_audio(f"{next_hour}")
        play_audio(period)
    else:
        play_audio(f"{hour}")
        play_audio("and")
        
        if 11 <= minute <= 19:
            play_audio(f"{minute}")
        else:
            tens = (minute // 10) * 10
            ones = minute % 10
            if tens > 0:
                play_audio(f"{tens}")
                if ones > 0:
                    play_audio("and")
                    play_audio(f"{ones}")
            else:
                play_audio(f"{ones}")

        play_audio(period)

# Time-telling logic for Japanese using basic hour and minute rules
def play_time_audio_jp(hour, minute, period):
    play_audio("imaha")  # 今は (now)

    if period == "am":
        play_audio("am")  # 午前
    else:
        play_audio("pm")  # 午後

    play_audio(f"hour_{hour}")

    if minute == 0:
        play_audio("desu") # です (it's)
        return

    if minute in [10, 20, 30, 40, 50]:
        play_audio(f"minute_{minute}")
        play_audio("desu")
        return

    tens = (minute // 10) * 10
    ones = minute % 10

    if tens > 0:
        play_audio(f"minute_{tens}a")

    if ones > 0:
        play_audio(f"minute_{ones}")

    play_audio("desu")

# Function to announce the current system time
def announce_current_time(app):
    current_time = time.localtime()
    hour_24 = current_time.tm_hour
    minute = current_time.tm_min
    
    hour = hour_24 % 12 or 12

    if hour_24 >= 12:
        period = "pm"
    else:
        period = "am"



    if selected_language == "EN":
        play_time_audio_en(hour, minute, period)
    elif selected_language == "FA":
        play_time_audio_fa(hour, minute, period)
    elif selected_language == "JP":
        play_time_audio_jp(hour, minute, period)

# Function to announce a custom time entered by the user
def announce_custom_time(app, custom_hour, custom_minute, period):
    hour = custom_hour % 12 or 12


    if selected_language == "EN":
        play_time_audio_en(hour, custom_minute, period)
    elif selected_language == "FA":
        play_time_audio_fa(hour, custom_minute, period)
    elif selected_language == "JP":
        play_time_audio_jp(hour, custom_minute, period)

# GUI for the talking clock
class TalkingClockApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Talking Clock")

        # Digital clock display
        self.time_label = Label(root, font=("Arial", 24))
        self.time_label.pack(pady=20)

        # Variables for language and voice selection with default EN Hiva
        self.language_var = StringVar(value="EN")
        self.voice_var = StringVar(value="Hiva")

        # Language selection dropdown
        Label(root, text="Select Language:").pack()
        self.language_menu = OptionMenu(root, self.language_var, *VOICE_PATHS.keys(), command=self.update_voice_options)
        self.language_menu.pack()

        # Voice selection dropdown
        Label(root, text="Select Voice:").pack()
        self.voice_menu = OptionMenu(root, self.voice_var, "")
        self.voice_menu.pack()

        # Announce system time button
        self.system_time_button = Button(root, text="Announce System Time", command=self.announce_time)
        self.system_time_button.pack(pady=10)

        # Announce custom time button
        self.custom_time_button = Button(root, text="Announce Custom Time", command=self.get_custom_time)
        self.custom_time_button.pack(pady=10)

        # Start updating the digital clock
        self.update_digital_clock()
        self.update_voice_options(self.language_var.get())  # Update voices on initialization

    def update_digital_clock(self):
        current_time = time.strftime("%I:%M:%S %p")
        self.time_label.config(text=current_time)
        self.root.after(1000, self.update_digital_clock)

    def update_voice_options(self, selected_language):
        # Update available voices based on selected language
        voices = VOICE_PATHS[selected_language].keys()
        menu = self.voice_menu["menu"]
        menu.delete(0, "end")  # Clear the current voice options

        for voice in voices:
            menu.add_command(label=voice, command=lambda v=voice: self.voice_var.set(v))

        self.voice_var.set(next(iter(voices)))  # Set default voice to the first in the list

    def get_custom_time(self):
        global selected_language, selected_voice, BASE_AUDIO_PATH

        # Set language and voice
        selected_language = self.language_var.get()
        selected_voice = self.voice_var.get()
        BASE_AUDIO_PATH = os.path.join(AUDIO_BASE_PATH, VOICE_PATHS[selected_language][selected_voice])

        # Get custom time input
        custom_hour = simpledialog.askinteger("Input", "Enter the hour (1-12):", minvalue=1, maxvalue=12)
        custom_minute = simpledialog.askinteger("Input", "Enter the minute (0-59):", minvalue=0, maxvalue=59)
        period = simpledialog.askstring("Input", "Enter 'am' or 'pm':").lower()

        if period not in ["am", "pm"]:
            messagebox.showerror("Invalid Input", "Please enter 'am' or 'pm'.")
            return

        announce_custom_time(self, custom_hour, custom_minute, period)

    def announce_time(self):
        global selected_language, selected_voice, BASE_AUDIO_PATH

        # Set language and voice
        selected_language = self.language_var.get()
        selected_voice = self.voice_var.get()
        BASE_AUDIO_PATH = os.path.join(AUDIO_BASE_PATH, VOICE_PATHS[selected_language][selected_voice])

        announce_current_time(self)


# Start the application
def start_app():
    root = Tk()
    app = TalkingClockApp(root)
    root.mainloop()

# Run the app
if __name__ == "__main__":
    start_app()


pygame 2.6.1 (SDL 2.28.4, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


2024-10-18 10:52:21.382 python[56559:1597518] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-18 10:52:21.382 python[56559:1597518] +[IMKInputSession subclass]: chose IMKInputSession_Legacy
